In [1]:
from astromodels import Band
from astromodels import TemplateModel
import numpy as np
import string
from scipy import constants as const
import progressbar
from time import sleep
import ipyparallel as ipp
rc = ipp.Client()

/home/simonste/.venv/3ml/local/lib/python2.7/site-packages/ipyparallel/client/client.py:459: RuntimeWarning: 
            Controller appears to be listening on localhost, but not on this machine.
            If this is true, you should specify Client(...,sshserver='you@necromancer')
            or instruct your controller to listen on an external IP.
  RuntimeWarning)


In [2]:
def modfactory(band,col):
    from astromodels import Band
    from astromodels import TemplateModel
    import numpy as np
    import string
    from scipy import constants as const
    import progressbar
    from time import sleep
    from astromodels import TemplateModelFactory
    import sys
    sys.path.append('/home/simonste/research/xrb/scripts/modules')
    from my_functions import modelRinner
    phases = np.arange(0,1.01,0.01)
    print band,col
    tmf = TemplateModelFactory('Xrb_HSESRinner%s'%band, 
                               'template for model with HS Edge Surface and Rinner- %s'%band, 
                               phases, 
                               ['Inkl','Dtemp','HSTemp','HSWidth',
                                'T_sec','distfac','Dsize','R_inner'])

    Inkl_grid = np.arange(60,79,2)
    Dtemp_grid = np.arange(1000,3200,300)
    #Dsize_grid = [0.8]
    HSTemp_grid = np.arange(3100,7200,2000)
    HSWidth_grid = np.arange(1,42,10)
    T_sec_grid = np.arange(3800,4700,200)
    distfac_grid = np.arange(-1,1.01,1)
    Dsize_grid = [0.725,0.825,0.925,0.999]
    R_inner_grid = [1000,3000,5000,7000,9000]
   
    tmf.define_parameter_grid('Inkl', Inkl_grid)
    tmf.define_parameter_grid('Dtemp', Dtemp_grid)
    tmf.define_parameter_grid('HSTemp', HSTemp_grid)
    tmf.define_parameter_grid('HSWidth', HSWidth_grid)
    tmf.define_parameter_grid('T_sec', T_sec_grid)
    tmf.define_parameter_grid('distfac', distfac_grid)
    tmf.define_parameter_grid('Dsize', Dsize_grid)
    tmf.define_parameter_grid('R_inner', R_inner_grid)

    for a in Inkl_grid:
        for b in Dtemp_grid:
            for c in HSTemp_grid:
                for d in HSWidth_grid:
                    for e in T_sec_grid:
                        for f in distfac_grid:
                            for g in Dsize_grid:
                                    for h in R_inner_grid:
                                        tmf.add_interpolation_data(modelRinner(phases,
                                                                               a,b,c,d,
                                                                               e,f,g,
                                                                               h,col),
                                                       Inkl = a , Dtemp = b ,HSTemp = c, 
                                                                   HSWidth = d, T_sec =e,
                                                                   distfac = f, Dsize =g,
                                                                   R_inner = h)
    tmf.save_data(overwrite=True)
    print 'file saved'

In [3]:
dview = rc[:]
dview.block=True
filter_list = ['gband','rband','iband','zband','Jband','Hband','Kband']
#filter_list = ['iband','zband','Jband','Hband','Kband']
col_list = [0,1,2,3,4,5,6]
dview.map_sync(modfactory,filter_list,col_list) #col 1 = 0 in collist because 1 is added later

[None, None, None, None, None, None, None]